In [1]:
from tqdm import tqdm

In [2]:
!ls

1		     make_host_tables.ipynb  test_host.csv	       url.data
2		     predict.ipynb	     test_query_features.tsv   url_host
catboost_info	     queries.tsv	     train_host.tsv
global_features.tsv  sample.csv		     train.marks.tsv
host.data	     sub_cat.txt	     train_query_features.tsv


In [3]:
from urllib.parse import urlparse, urlsplit
import tldextract
with open('url.data', 'r') as url_file, open('host.data', 'w') as host_file, open('url_host', 'w') as url_host_file:
    counter = 0
    host_dict = {}
    for line in tqdm(url_file):
        line = line.strip()
        splited = line.split('\t')
        url_id = int(splited[0])
        url = splited[1]
        tld = tldextract.extract(url)
        if tld.subdomain == '':
            subdomain = ''
        else:
            subdomain = tld.subdomain + '.'
        
        if tld.suffix == '':
            suffix = ''
        else:
            suffix = '.' + tld.suffix
        host = subdomain + tld.domain + suffix
        if host[:4] == 'www.':
            host = host[4:]
        if host not in host_dict:
            host_dict[host] = counter
            host_file.write(str(counter) + '\t' + host + '\n')
            url_host_file.write(str(url_id) + '\t' + str(counter) + '\n')
            counter += 1
        else:
            host_id = host_dict[host]
            url_host_file.write(str(url_id) + '\t' + str(host_id) + '\n')
#     break

582167it [00:06, 93368.62it/s] 


In [4]:
url_host_dict = {}
with open('url_host', 'r') as f:
    for line in tqdm(f):
        line = line.strip()
        splited = line.split('\t')
        url_id = splited[0]
        host_id = splited[1]
        url_host_dict[url_id] = host_id

582167it [00:00, 973087.65it/s]


In [5]:
with open('train.marks.tsv', 'r') as train_url_file, open('train_host.tsv', 'w') as train_host_file:
    for line in tqdm(train_url_file):
        line = line.strip()
        splited = line.split('\t')
        query_id = splited[0]
        url_id = splited[1]
        host_id = url_host_dict[url_id]
        train_host_file.write(query_id + '\t' + host_id + '\n')

202079it [00:00, 623312.35it/s]


In [6]:
with open('sample.csv', 'r') as test_url_file, open('test_host.csv', 'w') as test_host_file:
    line = test_url_file.readline()
    test_host_file.write(line)
    for line in tqdm(test_url_file):
        line = line.strip()
        splited = line.split(',')
        query_id = splited[0]
        url_id = splited[1]
        host_id = url_host_dict[url_id]
        test_host_file.write(query_id + ',' + host_id + '\n')

403971it [00:00, 640538.17it/s]
